In [1]:
import sys
sys.path.append(r'D:/projects/singletrader/')
# import singletrader
from singletrader.datautils.qlibapi.constructor.base import MultiFactor
import pandas as pd
import warnings
from singletrader.factortesting.factor_testing import FactorEvaluation
from pprint import pprint



import numpy as np
warnings.filterwarnings('ignore')
import plotly.express as px
from samples.double_sort_framework.workflow import bar_resample,MultiFactorTesting
from plotly.figure_factory import create_table
# from IPython.display import HTML 

[18804:MainThread](2023-03-07 09:38:30,775) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[18804:MainThread](2023-03-07 09:38:30,980) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[18804:MainThread](2023-03-07 09:38:30,980) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/database/.singletrader/qlib_data')}


auth success 


In [2]:
#合并数据示例
from pathlib import Path
# __file__ = r'D:\projects\singletrader\samples\double_sort_framework\workflow.ipynb'
# file = Path(__file__)
# parent_path = file.parent
# data_file = parent_path.__str__() + '/data/' + r'price_and_factor_data.csv'
# merged_data = pd.read_csv(data_file)
# merged_data['date'] = pd.to_datetime(merged_data['date']) #date字段需要datetime格式；
# merged_data = merged_data.set_index(['date','asset']) #必须为date,asset双重索引；
# merged_data = merged_data.dropna()
# merged_data.dropna().head()

fields = []
names = []

features = ['skew',	'distance',	'mom3M', 'turnover3M']
fields += ['$close','$open','$high','$low','$avg','$volume','$circulating_market_cap','$turnover_ratio','$money','1/$pe_ratio']
bars = ['close','open','high','low','avg','volume','market_cap','turnover_ratio','money','ep']
names += bars


# mf = MultiFactor(field=fields,name=names,start_date='2009-01-01',end_date='2022-12-31')
# data = mf._data.swaplevel(0,1)
# data.index = data.index.set_names(['date','asset'])

# data = bar_resample(data,frequency='M')

In [ ]:
#收益月偏度计算
skew = data.groupby('asset').apply(lambda x:x['close'].droplevel('asset').pct_change().resample('M').apply(lambda x:x.skew()))
skew = skew.stack().swaplevel(0,1)
skew.name = 'skew'

#
#stddev_diff
stddev_diff = data.groupby('asset').apply(lambda x:(x['close'].droplevel('asset').pct_change().resample('M').apply(lambda x:x.std())).diff())
stddev_diff = stddev_diff.stack().swaplevel(0,1)
stddev_diff.name = 'stddev_diff'

#年度最高价距离计算 1 - close/Max(high,252)
distance = data.groupby('asset').apply(lambda x:(1-x['close'] / x['high'].rolling(252).max()).droplevel('asset').resample('M').last())
distance =  distance.stack().swaplevel(0,1)
distance.name = 'distance'


#行情数据降至月频率
bar_monthly = bar_resample(data[bars],frequency='M')

#不同month动量
mom =  [bar_monthly.groupby('asset').apply(lambda x:x['close'].shift(i-1)/x['close'].shift(i)-1).droplevel(0) for i in range(1,13)]
mom = pd.concat(mom,axis=1)
mom.columns = ['mom%dM' % i for i in range(1,13)]

#不同month动量
Mom2M =  bar_monthly.groupby('asset').apply(lambda x:x['close']/x['close'].shift(2)-1).droplevel(0)
Mom2M.name = 'Mom2M'


#3month turnover
turnover3M =  bar_monthly.groupby('asset').apply(lambda x:x['turnover_ratio'].rolling(3).sum()).droplevel(0)
turnover3M.name = 'turnover3M'




#过去3 months成交额中位数
median_turnover= bar_monthly['money'].groupby('asset').apply(lambda x:x.rolling(3).median())
median_turnover.name = 'amount3M'

#数据合并对齐
merged_data = pd.concat([bar_monthly, skew, distance, mom, Mom2M,turnover3M,median_turnover,stddev_diff],axis=1)

In [3]:
# merged_data.to_csv(r'D:\projects\singletrader\samples\double_sort_framework\report3rd\third_data.csv')
merged_data = pd.read_csv(r'D:\projects\singletrader\samples\double_sort_framework\report3rd\third_data.csv')
merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data = merged_data.set_index(['date','asset'])

# data sample

In [4]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 846384 entries, (Timestamp('2009-01-31 00:00:00'), '000001.XSHE') to (Timestamp('2022-12-31 00:00:00'), '689009.XSHG')
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   close           508232 non-null  float64
 1   open            508232 non-null  float64
 2   high            508232 non-null  float64
 3   low             508232 non-null  float64
 4   avg             508232 non-null  float64
 5   volume          846384 non-null  float64
 6   market_cap      508237 non-null  float64
 7   turnover_ratio  507845 non-null  float64
 8   money           846384 non-null  float64
 9   ep              508189 non-null  float64
 10  skew            513698 non-null  float64
 11  distance        448420 non-null  float64
 12  mom1M           503194 non-null  float64
 13  mom2M           498317 non-null  float64
 14  mom3M           493464 non-null  float64
 15  mom4M      

# 1.different momentum

In [5]:
merged_data = merged_data.dropna()

In [6]:
def ic_analysis(fe,**kwargs):
    def _ic_analysis(fe,**kwargs):
        ic_df = fe.get_factor_ics(method='normal',**kwargs)
        rankic_df = fe.get_factor_ics(method='rank',**kwargs)
        result_df = pd.DataFrame(columns = ic_df.columns)
        result_df.loc['ic.mean'] = ic_df.mean()
        result_df.loc['ic.ir'] = ic_df.mean() /  ic_df.std()
        result_df.loc['ic.t-stats'] = ic_df.mean() /  ic_df.std() * ((ic_df.__len__()/12) ** 0.5)
        # result_df.loc['rankic.mean'] = rankic_df.mean()
        # result_df.loc['rankic.ir'] = ic_df.mean() /  ic_df.std()
        # result_df.loc['rankic.t-stats'] = ic_df.mean() /  ic_df.std() * ((ic_df.__len__()/12) ** 0.5)

        return result_df
    ic_result = {}
    fe.winzorize()
    fe.standardize()
    ic_result['ic_raw'] = _ic_analysis(fe,**kwargs)
    fe.industry_neutralize()
    ic_result['ic_ind_neu'] =  _ic_analysis(fe,**kwargs)
    ic_result =  pd.concat(ic_result)
    return ic_result

    
    

- ic test

In [9]:
fe = FactorEvaluation(merged_data[bars],merged_data[['mom%dM' % i for i in range(1,13)]],freq=12)
ic_ana = ic_analysis(fe,add_shift=0,base='close-open')
ic_ana


mom1M_1M  mom2M_1M  mom3M_1M  mom4M_1M  mom5M_1M  \
ic_raw     ic.mean    -0.039399 -0.025225 -0.003760  0.002217 -0.003429   
           ic.ir      -0.340330 -0.246935 -0.042260  0.026266 -0.039796   
           ic.t-stats -1.223137 -0.887477 -0.151881  0.094399 -0.143027   
ic_ind_neu ic.mean    -0.046389 -0.023245 -0.006234  0.000009 -0.002668   
           ic.ir      -0.473895 -0.290938 -0.092156  0.000152 -0.042772   
           ic.t-stats -1.703166 -1.045623 -0.331208  0.000547 -0.153720   

                       mom6M_1M  mom7M_1M  mom8M_1M  mom9M_1M  mom10M_1M  \
ic_raw     ic.mean     0.003759  0.009115  0.006976  0.002733  -0.011311   
           ic.ir       0.042931  0.122569  0.083234  0.037512  -0.140185   
           ic.t-stats  0.154293  0.440509  0.299142  0.134818  -0.503820   
ic_ind_neu ic.mean     0.001756  0.005627  0.003429  0.000378  -0.008984   
           ic.ir       0.029371  0.108967  0.058276  0.007497  -0.155148   
           ic.t-stats  0.105559  0.391624  0.209442  0.026944  -0.557597   

                       mom11M_1M  mom12M_1M  
ic_raw     ic.mean      0.011199   0.009796  
           ic.ir        0.157093   0.149051  
           ic.t-stats   0.564590   0.535686  
ic_ind_neu ic.mean      0.003528   0.006945  
           ic.ir        0.069532   0.157366  
           ic.t-stats   0.249895   0.565571

In [10]:
ic_merge = ic_ana[ic_ana.index.get_level_values(1)=='ic.mean'].droplevel(1).T
px.bar(ic_merge,barmode='group',title='ic.mean of different momentum factor')

## Mom2M

In [11]:
liquidity_group = merged_data.groupby(level=0).apply(lambda x:pd.qcut(x['amount3M'],5,labels=['low','moderate_low','medium','moderate_high','high'])).droplevel(0)
ep_group = merged_data.groupby(level=0).apply(lambda x:pd.qcut(x['ep'],5,labels=['low','moderate_low','medium','moderate_high','high'])).droplevel(0)

In [ ]:
def factor_ana(factor):
    def get_ep_liq_group_ic(factor,group_df=ep_group):
        ics = {}
        for group in ['low','moderate_low','medium','moderate_high','high']:
            ics[group] = ic_analysis(FactorEvaluation(bar_data=merged_data[bars],factor_data=merged_data[group_df==group][factor],freq=12), add_shift=0,base='close-open')
        ics['total'] = ic_analysis(FactorEvaluation(bar_data=merged_data[bars],factor_data=merged_data[factor],freq=12), add_shift=0,base='close-open')
        ics = pd.concat(ics)
        # ics = ics.reindex(['total'] + ['low','moderate_low','medium','moderate_high','high'],axis=1)
        return ics
    fe = FactorEvaluation(bar_data=merged_data[bars],factor_data=merged_data[[factor]],freq=12)
    gp = fe.get_group_returns(groups=5,add_shift=0,base='close-open')
    # print(gp.head())
    line1 = px.line((1+gp.unstack()).droplevel(0,axis=1).cumprod(),title='different group nv')
    line1.show()

    from singletrader.performance.common import performance_indicator
    PI = np.round(performance_indicator((1+gp.unstack()).droplevel(0,axis=1).cumprod(),freq=12,language='en'),4)
    table = create_table(PI.reset_index())
    table.show()

    all_ic = ic_analysis(fe,periods=(1,2,3,4,5,6,7,8,9,10,11,12),base='close-open',add_shift=0,total=False)
    tabel1=create_table(round(all_ic,4).reset_index())
#     HTML(tabel1.to_html())
    tabel1.show()
    
    ic_merge = all_ic[all_ic .index.get_level_values(1)=='ic.mean'].droplevel(1).T
    
    bar1 = px.bar(ic_merge,barmode='group',title='ic.mean of defferent lags')
    # HTML(bar1.to_html())
    bar1.show()
# 

    t = get_ep_liq_group_ic(factor)
    bar2 = px.bar(t[t.index.get_level_values(2)=='ic.mean'].droplevel(2).unstack().droplevel(0,axis=1),barmode='group',title='avg.ic of different ep groups')
    # HTML(bar2.to_html())
    bar2.show()
    
    t = get_ep_liq_group_ic(factor,group_df=liquidity_group)
    bar3 = px.bar(t[t.index.get_level_values(2)=='ic.mean'].droplevel(2).unstack().droplevel(0,axis=1),barmode='group',title='avg.ic of different liquidity groups')
    # HTML(bar3.to_html())
    bar3.show()



In [17]:
factor_ana('Mom2M')

# skew


In [ ]:
factor_ana('skew')

# distance

In [ ]:
factor_ana('distance')

# turnover3M

In [ ]:
factor_ana('turnover3M')

# stddev_diff

In [ ]:
factor_ana('stddev_diff')